In [21]:
import pandas as pd
import os
import numpy as np
from wordcloud import WordCloud
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [22]:
df = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/train_data/train.csv', header=None)
# df_test = pd.read_csv('/Users/marikhomeriki/code/marikhomeriki/product_review_analysis/raw_data/test_data/test.csv', header=None)
# df_test = df.rename({0: 'label', 1: 'text'}, axis = 1)
df = df.rename({0: 'label', 1: 'text'}, axis = 1)

In [23]:
df.head()

,label,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [24]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
    lowercased = text.lower() # Lower Case
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    return lemmatized

In [30]:
df_text = df.text

In [31]:
df_text = pd.DataFrame(df_text)

In [32]:
df_text

,text
0,"Unfortunately, the frustration of being Dr. Go..."
1,Been going to Dr. Goldberg for over 10 years. ...
2,I don't know what Dr. Goldberg was like before...
3,I'm writing this review to give you a heads up...
4,All the food is great here. But the best thing...
...,...
559995,Ryan was as good as everyone on yelp has claim...
559996,Professional \nFriendly\nOn time AND affordabl...
559997,Phone calls always go to voicemail and message...
559998,Looks like all of the good reviews have gone t...


In [33]:
len_train = int(10/100*len(df_text['text']))
df_text = df_text['text']
df_text = df_text[:len_train]

In [36]:
df_text = pd.DataFrame(df_text)

In [37]:
df_text['clean_text'] = df_text.text.apply(clean)
df_text['clean_text'] = df_text['clean_text'].astype('str')

In [38]:
df_text

,text,clean_text
0,"Unfortunately, the frustration of being Dr. Go...","['unfortunately', 'frustration', 'dr', 'goldbe..."
1,Been going to Dr. Goldberg for over 10 years. ...,"['going', 'dr', 'goldberg', 'year', 'think', '..."
2,I don't know what Dr. Goldberg was like before...,"['know', 'dr', 'goldberg', 'like', 'moving', '..."
3,I'm writing this review to give you a heads up...,"['writing', 'review', 'give', 'head', 'see', '..."
4,All the food is great here. But the best thing...,"['food', 'great', 'best', 'thing', 'wing', 'wi..."
...,...,...
55995,I took a party of 6 friends and co-workers the...,"['took', 'party', 'friend', 'co', 'worker', 'l..."
55996,Stop by for lunch based on Yelp reviews. Tasty...,"['stop', 'lunch', 'based', 'yelp', 'review', '..."
55997,Best persian food ever.\nWOW!\n\nPossibly the ...,"['best', 'persian', 'food', 'ever', 'nwow', 'n..."
55998,"Habibbbbbb is the rudest, meanest, uglyist, gu...","['habibbbbbb', 'rudest', 'meanest', 'uglyist',..."


In [40]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

data_vectorized = vectorizer.fit_transform(df_text['clean_text'])

lda_model = LatentDirichletAllocation(n_components=4)

lda_vectors = lda_model.fit_transform(data_vectorized)

In [176]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-5 - 1:-1]])

In [177]:
print_topics(lda_model, vectorizer)

Topic 0:
[('food', 63.17431405324073), ('good', 53.40349980476475), ('great', 42.77701187249921), ('place', 27.43256413107815), ('friendly', 27.246354006460017)]
Topic 1:
[('place', 54.46623574602995), ('service', 34.73973444795151), ('go', 31.877810293199804), ('back', 25.9231121538039), ('star', 18.24619548584007)]
Topic 2:
[('back', 30.679256679210262), ('time', 23.9372376271045), ('service', 18.8539950948999), ('place', 15.160167522936261), ('never', 15.037490507307602)]
Topic 3:
[('food', 42.33094780523769), ('good', 28.617843425717414), ('service', 17.06729557884156), ('better', 16.23645371788231), ('way', 16.19113143368664)]
